# Text Generation using NLP

## Importing the libraries and the dataset

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential, load_model


path_to_file = './shakespeare.txt'
text = open(path_to_file, 'r').read()

In [2]:
print(text[:1375])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

## Text Processing

In [3]:
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

84
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


In [4]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [5]:
ind_to_char = np.array(vocab)
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [6]:
encoded_text = np.array([char_to_ind[c] for c in text])
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [7]:
encoded_text.shape

(5445609,)

## Batch Creation

In [8]:
lines = '''From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,'''

len(lines)

131

In [9]:
seq_len = 135
total_num_seq = len(text)//(seq_len+1)
total_num_seq

40041

In [10]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
type(char_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [11]:
def create_seq_targets(seq):
  input_txt = seq[:-1] # Hello my nam
  target_txt = seq[1:] # ello my name
  return input_txt, target_txt

sequences = char_dataset.batch(seq_len+1, drop_remainder = True)
dataset = sequences.map(create_seq_targets)

for input_txt, target_txt in dataset.take(1):
  print(input_txt.numpy())
  print("".join(ind_to_char[input_txt.numpy()]))
  print('\n')
  print(target_txt.numpy())
  print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75
  1 56 74  1 75 63 60  1 73 64 71 60 73  1 74]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper s


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1
 56 74  1 75 63 60  1 73 64 71 60 73  1 74 63]
                     1
  From fairest 

In [12]:
batch_size = 128

buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder =True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 135), dtype=tf.int64, name=None), TensorSpec(shape=(128, 135), dtype=tf.int64, name=None))>

## Model Creation

In [13]:
 vocab_size = len(vocab)
 embed_dim = 64 ## Should be within the range of the vocab_size
 rnn_neurons = 1026
 vocab_size

84

In [14]:
def sparse_cat_loss(y_true, y_pred) :
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits = True)
  ## from_logits relates to the fact whether the data is one-hot encoded or not

In [15]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
  model=Sequential()
  model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
  model.add(GRU(rnn_neurons, return_sequences = True, stateful = True, recurrent_initializer = 'glorot_uniform'))
  model.add(Dense(vocab_size))

  model.compile('adam', loss = sparse_cat_loss)

  return model

In [16]:
model = create_model(vocab_size, embed_dim = embed_dim, rnn_neurons = rnn_neurons, batch_size = batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
epochs = 30
model.fit(dataset, epochs=epochs)

In [17]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size = 1)
model.load_weights('./shakespeare_gen.h5')
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters.
  '''

  num_generate = gen_size     # Number of characters to generate

  input_eval = [char_to_ind[s] for s in start_seed]    # Vecotrizing starting seed text

  input_eval = tf.expand_dims(input_eval, 0)      # Expand to match batch format shape

  text_generated = []      # Empty list to hold resulting generated text

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected

  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [20]:
print(generate_text(model, "From fairest creatures we desire increase, That thereby beauty's rose might never die, But as the riper should by time decease, His tender heir might bear his memory:", gen_size = 1000))

From fairest creatures we desire increase, That thereby beauty's rose might never die, But as the riper should by time decease, His tender heir might bear his memory:  
    If this imperkinate Luceltion strong,
    Shake off admitted to thee;  
    Sir Valentine with all despite of chin.
  Nurse. O fellow Tribunes, gentle journey, boy!' O, where
    Help, Jessica, sir? Titus well methought;
    Be you the most wit's that, it wonders to't.
  ULYSSES. Most  my soul, I take like to the band;
    And humbly yet, by the distance of your
    lapins in their loud'sh; what if it be, grant me the waist
    of truth hath not her course in the mines of a king
    Did grieve MALVOLIO here done so slight.
  SPEED. Why, I pray you? Do all come hither to abhorred affairs,  
    and will be woo'd, we mean to speak with your head, and
    we'll followed the leading of's. Portia, what!
    The thick end or of us she miss the lady chokes
    To grace us?
  DUKE. It done, you were my wood.               E